In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [3]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1993
).filter(
    pl.col("komora_komplet").str.contains('Poslanecká') | pl.col('komora_komplet').str.contains('PČR, PS')
).with_columns(
    pl.col('mluvci_id').str.split('/sqw/detail.sqw?id=').alias('id')
).with_columns(
    pl.when(pl.col('id').len() > 2).then(pl.col('id').list.slice(1)).alias('id')
).explode(
    "id"
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov") # tady budou zlobit dvojité mezery, pohlídat při čištění
).sort(
    by="datum"
).with_columns(
    pl.when(pl.col("soubor") == "1996ps_stenprot_006schuz_s006024.htm").then(pl.lit(2006)).otherwise(pl.col('obdobi')).alias('obdobi')
).with_columns(
    pl.col("text").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_text")
).with_columns(
    pl.col("mluvci").str.split("(").list.slice(1).list.eval(
        pl.element().str.split(')').list.first()
    ).list.eval(
        pl.element().str.to_lowercase().str.strip_chars(), parallel=True
    ).alias("poznamky_mluvci")
).drop_nulls(
    subset=['datum']
).collect()

In [199]:
df.sample(10)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Poslanec Jaroslav Krákora""","""2006ps_stenprot_013schuz_s013184.htm""","""/sqw/detail.sqw?id=5279""","""Nyní bychom měli hlasovat pozměňovacích návrzích pana poslance Kochana označených bodem C3a a C3c, kde navrhuje zpětvzetí tohoto návrhu.""",20,"""ps""",2006,13,184,2007-03-21,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""13. schůze""",true,true,true,"""5279""",18,[],[]
"""Místopředsedkyně PSP Miroslava Němcová""","""2002ps_stenprot_055schuz_s055196.htm""","""/sqw/detail.sqw?id=401""","""Děkuji vám, pane místopředsedo. Zahajuji všeobecnou rozpravu. Písemnou přihlášku nemám. Táži se tedy na vaše přihlášky nyní. Nikdo se do všeobecné roz…",6,"""ps""",2002,55,196,2006-04-26,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""55. schůze""",true,true,true,"""401""",56,[],[]
"""Poslanec Miroslav Opálka""","""2013ps_stenprot_035schuz_s035054.htm""","""/sqw/detail.sqw?id=5322""","""V dalším kroku budeme hlasovat o návrhu C, což je můj návrh - 1 200 korun jednorázový příspěvek. Garanční výbor nedoporučuje.""",9,"""ps""",2013,35,54,2015-11-13,"""Parlament České republiky, Poslanecká sněmovna 2013 – 2017""","""35. schůze""",true,true,true,"""5322""",20,[],[]
"""Poslanec Michal Kraus""","""2002ps_stenprot_051schuz_s051231.htm""","""/sqw/detail.sqw?id=77""","""B/260. Nesouhlas. (Ministr: Nesouhlas.)""",26,"""ps""",2002,51,231,2005-12-02,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""51. schůze""",true,true,true,"""77""",3,"[""ministr: nesouhlas.""]",[]
"""Místopředseda vlády ČR Ivan Kočárník""","""1996ps_stenprot_007schuz_s007033.htm""","""/sqw/hlasy.sqw?G=5978""","""Pokud jde o první hlasování, které pro pana poslance Kroupu dopadlo dobře, to šlo o přesun z vládní rozpočtové rezervy na jiný výdaj. Tam se nemění an…",60,"""ps""",1996,7,33,1996-12-13,"""Parlament České republiky, Poslanecká sněmovna 1996 – 1998""","""7. schůze""",true,true,true,null,48,[],[]
"""Předseda PSP Lubomír Zaorálek""","""2002ps_stenprot_027schuz_s027153.htm""","""/sqw/detail.sqw?id=252""","""Táži se, zda se někdo další hlásí do obecné rozpravy. Pokud ne, obecnou rozpravu končím. Je prostor pro závěrečné slovo. Předpokládám, že to je to, na…",6,"""ps""",2002,27,153,2004-02-17,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""27. schůze""",true,true,true,"""252""",31,[],[]
"""Poslanec Michal Kučera""","""2021ps_stenprot_001schuz_s001043.htm""","""/sqw/detail.sqw?id=6216""","""Dobrý večer, dámy a pánové. Vážený pane předsedo volební komise, prostřednictvím paní předsedající, já tedy věřím, že jste předseda volební komise, pr…",3,"""ps""",2021,1,43,2021-11-10,"""PČR, PS 2021-...""","""1. schůze""",true,true,true,"""6216""",217,[],[]
"""Poslanec Michal Doktor""","""2006ps_stenprot_007schuz_s007043.htm""","""/sqw/detail.sqw?id=356""","""Pane předsedající, po dohodě s předsedou rozpočtového výboru bych chtěl navrhnout jako hlasovatelný návrh zkrácení lhůty k projednávání na 48 hodin. T…",5,"""ps""",2006,7,43,2006-11-29,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""7. schůze""",true,true,true,"""356""",25,[],[]
"""Místopředseda PSP Lubomír Zaorálek""","""2010ps_stenprot_032schuz_s032265.htm""","""/sqw/detail.sqw?id=252""","""Paní poslankyně, jenom je tu takový návrh. Já vidím z gest poslanců, že oni ten písemný návrh mají před sebou, tudíž netrvají na tom, abyste ho v celé…",10,"""ps""",2010,32,265,2011-12-15,"""Parlament České republiky, Poslanecká sněmovna 2010 – 2013""","""32. schůze""",true,true,true,"""252""",77,[],[]


In [22]:
df.group_by("pocet_slov").len().sort(by="pocet_slov")

pocet_slov,len
u32,u32
0,1353
1,7804
2,3233
3,4402
4,9248
5,7185
6,7199
7,7759
8,8472


In [24]:
df.select(pl.col("pocet_slov")).median()

pocet_slov
f64
45.0


In [34]:
df.select(pl.col("pocet_slov")).quantile(0.25)

pocet_slov
f64
18.0


In [18]:
slov_po_roce = df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('pocet_slov').sum())

In [5]:
len(df)

585180

In [65]:
df.columns

['mluvci',
 'soubor',
 'mluvci_id',
 'text',
 'poradi',
 'komora',
 'obdobi',
 'schuze',
 'cast',
 'datum',
 'komora_komplet',
 'schuze_komplet',
 'prepsano',
 'autorizovano',
 'mluvci_jisty',
 'id',
 'pocet_slov',
 'poznamky_text',
 'poznamky_mluvci']

In [191]:
def zavorka(retezec, doslova=True, frejm=df, epocha="datum"):

    frejm = frejm.with_columns(
        (pl.col('komora').cast(str) + pl.lit('-') + pl.col('obdobi').cast(str) + pl.lit('-') + pl.col('schuze').cast(str) + pl.lit('-') + pl.col('cast').cast(str) + pl.lit('-') + pl.col('poradi').cast(str)).alias('id')
    )

    prislusne_projevy = frejm.explode("poznamky_text").explode("poznamky_mluvci").filter(
            pl.col('poznamky_text').str.contains(retezec, literal=doslova) 
        |
            pl.col('poznamky_mluvci').str.contains(retezec, literal=doslova)
        ).unique(
        subset=['datum','cast','poradi']
        ).sort(by='datum')
    
    if epocha == "datum":
        
        projevu_celkem = frejm.sort(by='datum').group_by_dynamic(index_column='datum',every='1y').agg(pl.col("id").unique().len())
        
        prislusne_projevy = prislusne_projevy.group_by_dynamic(
            index_column='datum',every='1y'
        ).agg(
            pl.col("id").unique().len()
        )

    elif epocha == "obdobi":

        projevu_celkem = frejm.group_by("obdobi").agg(pl.col("id").unique().len())
        prislusne_projevy = prislusne_projevy.group_by(
            "obdobi"
        ).agg(
            pl.col("id").unique().len()
        )

    return prislusne_projevy.join(
        projevu_celkem, how="right",on=epocha
    ).with_columns(
        (pl.col("id") / pl.col("id_right")).alias("podil")
    ).sort(by=epocha)

In [197]:
zavorka('smích',epocha='datum')

id,datum,id_right,podil
u32,date,u32,f64
38,1993-01-01,9366,0.004057
21,1994-01-01,7771,0.002702
37,1995-01-01,9237,0.004006
53,1996-01-01,9572,0.005537
48,1997-01-01,16632,0.002886
54,1998-01-01,11831,0.004564
91,1999-01-01,20885,0.004357
79,2000-01-01,24941,0.003167
84,2001-01-01,20971,0.004006


In [7]:
'; '.join(df.explode("poznamky_text").group_by(
    "poznamky_text"
).len(
).sort(
    by='len',
    descending=True
).head(500).filter(pl.col('poznamky_text').is_not_null()).select(pl.col('poznamky_text')).to_series().to_list())

'potlesk.; ?; souhlas.; ano.; nesouhlasné.; nesouhlas.; potlesk zprava.; hluk v sále.; ministr: nesouhlas.; slibuji.; nesrozumitelné; předsedající: čas.; smích v sále.; ministr: souhlas.; potlesk poslanců čssd.; potlesk poslanců ods.; potlesk zleva.; upozornění na čas; potlesk poslanců ksčm.; potlesk v části sálu.; souhlasné.; potlesk části poslanců.; předsedající upozorňuje na čas.; 14.40 hodin; 14.50 hodin; ne.; 15.10 hodin; 15.20 hodin; 15.00 hodin; 15.40 hodin; upozornění na čas.; 15.30 hodin; 15.50 hodin; velký hluk v sále.; ministr nedoporučuje.; v sále je velký hluk.; 16.10 hodin; smích.; potlesk z lavic ano.; 16.20 hodin; potlesk poslanců ano.; neutrální.; 17.10 hodin; 16.00 hodin; 17.20 hodin; pobavení v sále.; 16.40 hodin; 17.30 hodin; 16.30 hodin; 10.20 hodin; 10.10 hodin; 11.00 hodin; 17.50 hodin; 10.50 hodin; 16.50 hodin; 11.40 hodin; 11.20 hodin; 17.40 hodin; 14.30 hodin; 11.10 hodin; 11.30 hodin; 10.40 hodin; 10.30 hodin; 11.50 hodin; jednání zahájeno v 9.00 hodin.; 18.0

In [217]:
def graf_vyvoje(slovo, epocha="datum"):
    if epocha == "datum":
        osa_x = 'datum:T'
    elif epocha == "obdobi":
        osa_x = "obdobi:N"
    return alt.Chart(
        zavorka(slovo,epocha=epocha).to_pandas(),
        width = 800
    ).mark_bar(
    ).encode(
        alt.X(osa_x),
        alt.Y("podil:Q")
    )

In [231]:
graf_vyvoje("smích",epocha="datum")

alt.Chart(...)

In [229]:
graf_vyvoje("hluk",epocha="obdobi")

alt.Chart(...)

In [227]:
graf_vyvoje("bučení",epocha="obdobi")

alt.Chart(...)

In [225]:
graf_vyvoje("smích",epocha="obdobi")

alt.Chart(...)

In [223]:
graf_vyvoje("potlesk",epocha="obdobi")

alt.Chart(...)

In [9]:
alt.Chart(
    zavorka("smích").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [10]:
alt.Chart(
    zavorka("výkřiky").to_pandas(),
    width = 800
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [11]:
df.filter(pl.col("text").str.contains("[Bb]učení"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Ministr zdravotnictví ČR David Rath""","""2002ps_stenprot_055schuz_s055136.htm""","""http://wtd.vlada.cz/scripts/detail.php?id=9214""","""Děkuji. Je to myslím docela zase signifikantní a typické, když názor vyslovený těmi, kterých se to bytostně týká, tedy lékaři zaměstnanci a zdravotním…",15,"""ps""",2002,55,136,2006-04-21,"""Parlament České republiky, Poslanecká sněmovna 2002 – 2006""","""55. schůze""",true,true,true,null,201,[],[]
"""Poslanec David Rath""","""2006ps_stenprot_027schuz_s027049.htm""","""/sqw/detail.sqw?id=5406""","""Dámy a pánové, pane předsedající, já bych prosil, abyste poučil o slušném chování poslance ODS, aby zde nebučeli. Nejsou, jak jsou zvyklí, v chlívě. Z…",3,"""ps""",2006,27,49,2008-01-30,"""Parlament České republiky, Poslanecká sněmovna 2006 – 2010""","""27. schůze""",true,true,true,"""5406""",426,[],[]
"""Místopředseda PSP Vojtěch Pikal""","""2017ps_stenprot_040schuz_s040102.htm""","""/sqw/detail.sqw?id=6531""","""Děkuji za slovo. Protože nemáme tu britskou tradici, tak tady neprobíhalo během předchozích projevů bučení a podobně, místo toho všeobecný nezájem pří…",7,"""ps""",2017,40,102,2020-01-23,"""Parlament České republiky, Poslanecká sněmovna 2017 – 2021""","""40. schůze""",true,true,true,"""6531""",272,"[""potlesk poslanců pirátů.""]",[]
"""Předsedkyně PSP Markéta Pekarová Adamová""","""2021ps_stenprot_005schuz_s005019.htm""","""/sqw/detail.sqw?id=6147""","""Vzhledem k tomu, že program schůze nebyl schválen a nikdo další se s přednostním právem nehlásí, tak já tuto 5. schůzi Poslanecké sněmovny končím. (Po…",2,"""ps""",2021,5,19,2021-12-15,"""PČR, PS 2021-...""","""5. schůze""",true,true,true,"""6147""",77,"[""poslanci tlučou do lavic, ozývá se bučení a volání."", ""schůze skončila ve 12.14 hodin.""]",[]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]
"""Poslanec Jakub Michálek""","""2021ps_stenprot_055schuz_s055007.htm""","""/sqw/detail.sqw?id=6477""","""Děkuji za slovo. Vážená paní předsedkyně, vážené kolegyně, vážení kolegové, pojďme prosím nechat v rozpravě vystoupit i ministry, například tady pana …",9,"""ps""",2021,55,7,2023-02-28,"""PČR, PS 2021-...""","""55. schůze""",true,true,true,"""6477""",148,"[""nesouhlas a bučení poslanců ano.""]",[]


In [12]:
alt.Chart(
    zavorka("bučen").to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [13]:
df.filter(pl.col("text").str.contains("(Buče",literal=True))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]
"""Místopředseda PSP Jan Skopeček""","""2021ps_stenprot_009schuz_s009170.htm""","""/sqw/detail.sqw?id=6418""","""Děkuji za slovo. Vážené kolegyně, vážení kolegové, prostřednictvím paní předsedající bych chtěl reagovat na pana předsedu Okamuru. Jsem přesvědčen, že…",3,"""ps""",2021,9,170,2022-02-18,"""PČR, PS 2021-...""","""9. schůze""",true,true,true,"""6418""",119,"[""bučení v lavicích spd."", ""potlesk z řad koalice.""]",[]


In [14]:
alt.Chart(
    zavorka("posmě[cš]",doslova=False).to_pandas(),
    width = 250
).mark_bar(
).encode(
    alt.X('datum:T'),
    alt.Y('podil:Q')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [15]:
df.filter(
    pl.col("datum").dt.year().is_between(1940,1970)
).filter(
    pl.col("text").str.contains("(Smích",literal=True)
)

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,list[str],list[str]


In [16]:
alt.Chart(
    pl.concat([zavorka("smích"), zavorka("potlesk"), zavorka("křik")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [36]:
alt.Chart(
    pl.concat([zavorka("potlesk zleva"), zavorka("potlesk zprava")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [37]:
def obrat(cislo):
    if cislo >= 1:
        return cislo
    if cislo < 1:
        return -(1/cislo)

In [40]:
obrat(0.1)

-10.0

In [41]:
pravolevo = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)",doslova=False
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['datum','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", doslova=False
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['datum','zprava'])
    ), on="datum", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
).filter(
    pl.col('datum').dt.year() >= 1993
)
pravolevo

datum,zleva,datum_right,zprava,pravo-levo
date,f64,date,f64,f32
1995-01-01,0.000325,1995-01-01,0.000108,-3.0
1996-01-01,0.000321,1996-01-01,0.000642,2.0
1997-01-01,0.001039,1997-01-01,0.000244,-4.25
1998-01-01,0.001107,1998-01-01,0.000511,-2.166667
1999-01-01,0.001766,1999-01-01,0.00211,1.194444
2000-01-01,0.000927,2000-01-01,0.001054,1.136364
2001-01-01,0.000296,2001-01-01,0.000741,2.5
2002-01-01,0.000371,2002-01-01,0.001237,3.333333
2003-01-01,0.000464,2003-01-01,0.003758,8.1


## Po 9 letech se víc tleská zleva!

In [43]:
alt.Chart(
    pravolevo.to_pandas(),
    title=["„Potlesk zprava“ (> 0) a „potlesk zleva“ (< 0)","ve stenozáznamech PS PSP ČR"]
).mark_bar().encode(alt.Y('datum:T'),alt.X('pravo-levo:Q',axis=alt.Axis(title="<– víc se tleská zleva | víc se tleská zprava ->")))

alt.Chart(...)

In [263]:
pravolevo_obdobi = zavorka(
    "(potlesk zleva|potlesk na levici|potlesk z levé|potlesk vlevo)", epocha="obdobi",doslova=False
).with_columns(
    pl.col("podil").alias("zleva")
).select(
    pl.col(['obdobi','zleva'])
).join(
    zavorka(
        "(potlesk zprava|potlesk na pravici|potlesk z pravé|potlesk vpravo)", epocha="obdobi", doslova=False
    ).with_columns(
        pl.col("podil").alias("zprava")
    ).select(
        pl.col(['obdobi','zprava'])
    ), on="obdobi", how="full"
).with_columns(
    (pl.col('zprava') / pl.col('zleva')).map_elements(obrat, return_dtype=pl.Float32).alias('pravo-levo')
)
pravolevo_obdobi

obdobi,zleva,obdobi_right,zprava,pravo-levo
i32,f64,i32,f64,f32
1993,0.0001,1993,0.000033,-3.0
1996,0.000876,1996,0.000454,-1.928571
1998,0.000905,1998,0.001128,1.246575
2002,0.000791,2002,0.003274,4.138462
2006,0.005086,2006,0.004946,-1.028169
2010,0.012582,2010,0.006162,-2.041958
2013,0.00415,2013,0.011317,2.727011
2017,0.001177,2017,0.007105,6.037383
2021,0.005042,2021,0.004077,-1.236686


In [265]:
alt.Chart(
    pravolevo_obdobi.to_pandas(),
    title=["„Potlesk zprava“ (> 0) a „potlesk zleva“ (< 0)","ve stenozáznamech PS PSP ČR"]
).mark_bar().encode(alt.Y('obdobi:N'),alt.X('pravo-levo:Q',axis=alt.Axis(title="<– víc se tleská zleva | víc se tleská zprava ->")))

alt.Chart(...)

## Počkat: kdo sedí vpravo, a kdo vlevo?

In [272]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Andrej Babiš'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

5

In [274]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Andrej Babiš'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

28

In [276]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Petr Fiala'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

8

In [278]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Petr Fiala'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

0

In [280]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Tomio Okamura'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

2

In [282]:
len(df.filter(
    (pl.col("obdobi") == 2021) & (pl.col('mluvci').str.contains('Tomio Okamura'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

10

In [284]:
len(df.filter(
    (pl.col('mluvci').str.contains('Grebeníček'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

13

In [286]:
len(df.filter(
    (pl.col('mluvci').str.contains('Grebeníček'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

0

In [288]:
len(df.filter(
    (pl.col('mluvci').str.contains('Miroslav Sládek'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

0

In [290]:
len(df.filter(
    (pl.col('mluvci').str.contains('Miroslav Sládek'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

9

In [292]:
len(df.filter(
    (pl.col('mluvci').str.contains('Bohuslav Sobotka'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zleva"))))

47

In [294]:
len(df.filter(
    (pl.col('mluvci').str.contains('Bohuslav Sobotka'))
).explode('poznamky_text').filter(pl.col('poznamky_text').str.contains(("(?i)potlesk zprava"))))

0

In [44]:
alt.Chart(
    pl.concat([zavorka("pobavení"), zavorka("veselost"), zavorka("veselí")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [45]:
alt.Chart(
    pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
<>:2: SyntaxWarning: invalid escape sequence '\('
C:\Users\micha\AppData\Local\Temp\ipykernel_25748\4207413198.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
C:\Users\micha\AppData\Local\Temp\ipykernel_25748\4207413198.py:2: SyntaxWarning: invalid escape sequence '\('
  pl.concat([zavorka("\(Hluk"), zavorka("\(Ticho")]).to_pandas(),
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [46]:
alt.Chart(
    pl.concat([zavorka("(Smích", doslova=True), zavorka("(Veselí", doslova=True), zavorka("(Pobavení", doslova=True)]).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('podil:Q', axis=alt.Axis(title=None)),
    alt.Row('co:N')
)

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [47]:
alt.Chart(
    df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len()).to_pandas(),
    width = 500, height = 100
).mark_bar(
).encode(
    alt.X('datum:T', axis=alt.Axis(title=None)),
    alt.Y('text:Q', axis=alt.Axis(title=None)),
)

alt.Chart(...)

In [48]:
df.group_by_dynamic(index_column='datum',every='1y').agg(pl.col('text').unique().len())

datum,text
date,u32
1993-01-01,9337
1994-01-01,7730
1995-01-01,9219
1996-01-01,9349
1997-01-01,16367
1998-01-01,11744
1999-01-01,20383
2000-01-01,23721
2001-01-01,20242


In [304]:
zebricek_potlesku = df.filter(
    pl.col("obdobi") == 2021
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("potlesk")
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

zebricek_smichu

mluvci,len
str,u32
"""Poslanec Andrej Babiš""",321
"""Poslanec Tomio Okamura""",91
"""Místopředseda PSP Karel Havlíček""",72
"""Poslanec Patrik Nacher""",66
"""Místopředsedkyně PSP Věra Kovářová""",52
"""Místopředseda PSP Jan Skopeček""",45
"""Poslankyně Berenika Peštová""",45
"""Poslanec Radek Vondráček""",38
"""Poslanec Jakub Michálek""",38


In [378]:
zebricek_smichu = df.filter(
    pl.col("obdobi") == 2021
).explode(
    "poznamky_text"
).filter(
    pl.col("poznamky_text").str.contains("smích") 
    | 
    pl.col("poznamky_text").str.contains("pobavení") 
    | 
    pl.col("poznamky_text").str.contains("veselí") 
    | 
    (~pl.col("poznamky_text").str.contains("smíchem"))
).group_by(
    "mluvci"
).len(
).sort(
    by="len",descending=True
)

zebricek_smichu

mluvci,len
str,u32
"""Poslanec Andrej Babiš""",2352
"""Místopředseda PSP Karel Havlíček""",1617
"""Místopředsedkyně PSP Věra Kovářová""",1575
"""Předsedkyně PSP Markéta Pekarová Adamová""",1534
"""Místopředseda PSP Jan Bartošek""",1474
"""Místopředsedkyně PSP Olga Richterová""",1439
"""Místopředseda PSP Jan Skopeček""",1318
"""Poslankyně Alena Schillerová""",1078
"""Místopředseda PSP Aleš Juchelka""",1038


In [380]:
zebricek_slov = df.filter(
    pl.col("obdobi") == 2021
).group_by(
    "mluvci"
).agg(
    pl.col("pocet_slov").sum()
)

zebricek_slov

mluvci,pocet_slov
str,u32
"""Předseda Národní rozpočtové rady Mojmír Hampl""",1055
"""Poslankyně Barbora Urbanová""",16309
"""Ministr životního prostředí ČR Richard Brabec""",3261
"""Předseda vlády ČR Andrej Babiš""",4345
"""Poslanec Pavel Svoboda""",5505
"""Poslanec Karel Tureček""",28239
"""Poslanec David Šimek""",8756
"""Poslanec Marek Výborný""",56762
"""Poslankyně Zdenka Němečková Crkvenjaš""",4834


In [382]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
1,"""Místopředseda PSP Tomio Okamura""",4,0.25
197,"""Poslanec Petr Fiala""",1043,0.188878
3,"""Poslanec Josef Váňa""",122,0.02459
348,"""Poslanec Josef Bernard""",23192,0.015005
6,"""Veřejný ochránce práv Stanislav Křeček""",769,0.007802
9,"""Markéta Pekarová Adamová""",1252,0.007188
3,"""Senátor Ladislav Václavec""",532,0.005639
20,"""Předsedající Radek Vondráček""",3591,0.005569
1038,"""Místopředseda PSP Aleš Juchelka""",188070,0.005519


In [402]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).filter(
    pl.col("pocet_slov") > 10000
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
348,"""Poslanec Josef Bernard""",23192,0.015005
1038,"""Místopředseda PSP Aleš Juchelka""",188070,0.005519
1575,"""Místopředsedkyně PSP Věra Kovářová""",300975,0.005233
1318,"""Místopředseda PSP Jan Skopeček""",261270,0.005045
1534,"""Předsedkyně PSP Markéta Pekarová Adamová""",305106,0.005028
1474,"""Místopředseda PSP Jan Bartošek""",313308,0.004705
1439,"""Místopředsedkyně PSP Olga Richterová""",384269,0.003745
100,"""Poslanec Jaroslav Foldyna""",28874,0.003463
45,"""Poslanec Vít Vomáčka""",13931,0.00323


In [404]:
df.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).select(pl.col("pocet_slov").sum())

pocet_slov
u32
347395


In [406]:
df.filter(pl.col("mluvci").str.contains("Ivan Bartoš")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("směj"))

mluvci,soubor,mluvci_id,text,poradi,komora,obdobi,schuze,cast,datum,komora_komplet,schuze_komplet,prepsano,autorizovano,mluvci_jisty,id,pocet_slov,poznamky_text,poznamky_mluvci
str,str,str,str,i32,str,i32,i32,i32,date,str,str,bool,bool,bool,str,u32,str,list[str]
"""Místopředseda vlády a ministr pro místní rozvoj ČR Ivan Bartoš""","""2021ps_stenprot_019schuz_s019175.htm""","""https://www.vlada.cz/cz/clenove-vlady/ivan-bartos-191704/""","""(Směje se.) Já jsem rád, že to takhle řešíme. Tak uděláme analýzu, které nejdůležitější legislativy kdy šly v předchozím volebním období poslaneckým n…",2,"""ps""",2021,19,175,2022-05-11,"""PČR, PS 2021-...""","""19. schůze""",true,true,true,null,91,"""směje se.""",[]


In [259]:
zebricek_smichu.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).filter(
    pl.col("len").is_null()
).sort(
    by="pocet_slov",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
null,"""Místopředseda vlády a ministr pro místní rozvoj ČR Ivan Bartoš""",163860,null
null,"""Poslanec Jiří Mašek""",162799,null
null,"""Místopředsedkyně PSP Jana Mračková Vildumetzová""",114475,null
null,"""Poslanec Vladimír Zlínský""",114388,null
null,"""Poslanec Igor Hendrych""",89885,null
null,"""Poslanec Zdeněk Kettner""",87667,null
null,"""Poslankyně Marie Pošarová""",70868,null
null,"""Poslankyně Klára Dostálová""",68324,null
null,"""Poslanec Radek Rozvoral""",64222,null


In [306]:
zebricek_potlesku.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).filter(
    pl.col("len").is_null()
).sort(
    by="pocet_slov",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
null,"""Poslanec Josef Kott""",53022,null
null,"""Ministr školství, mládeže a tělovýchovy ČR Mikuláš Bek""",49888,null
null,"""Poslanec David Štolpa""",17001,null
null,"""Ministr průmyslu a obchodu ČR Lukáš Vlček""",16854,null
null,"""Ministr pro místní rozvoj ČR Petr Kulhánek""",15043,null
null,"""Poslanec Matěj Ondřej Havel""",13488,null
null,"""Ministr školství, mládeže a tělovýchovy ČR Vladimír Balaš""",12668,null
null,"""Poslanec Petr Bendl""",12321,null
null,"""Poslanec Michael Rataj""",10725,null


In [320]:
zebricek_potlesku.join(
    zebricek_slov, how="right", on="mluvci"
).with_columns(
    (pl.col("len") / pl.col("pocet_slov")).alias("podil")
).drop_nulls(
).filter(
    pl.col("pocet_slov") > 31000
).sort(
    by="podil",descending=True
)

len,mluvci,pocet_slov,podil
u32,str,u32,f64
112,"""Poslanec Robert Králíček""",79673,0.001406
38,"""Poslankyně Renata Oulehlová""",31971,0.001189
592,"""Poslankyně Alena Schillerová""",511438,0.001158
33,"""Poslanec Milan Feranec""",34981,0.000943
44,"""Poslanec Marek Benda""",49888,0.000882
29,"""Poslanec Tomáš Kohoutek""",34246,0.000847
23,"""Ministr spravedlnosti ČR Pavel Blažek""",36491,0.00063
41,"""Poslanec Kamal Farhan""",67206,0.00061
147,"""Poslanec Radek Vondráček""",245689,0.000598


In [326]:
df.filter(pl.col("mluvci").str.contains("Robert Králíček")).explode("poznamky_text").filter(pl.col("poznamky_text").str.contains("potlesk")).group_by("text").len().sort(by="len")

text,len
str,u32
"""Já se bavím. Takže, přátelé, právě jsme se dozvěděli, že za to, že voliči hnutí STAN vykroužkovali sebe a dostali se sem Piráti, může Andrej Babiš, (S…",1
"""Abych na něco nezapomněl... (Dívá se do podkladů.) Potom statistika. My nejsme schopni ověřit - a už na výboru pro veřejnou správu - nejsme schopni ov…",1
"""Mně to přijde ubohé, pane ministře, se tu hádat o panu Šteffelovi. Pan Šteffel na MMR nic nespáchal. To spáchal až na Dopravním podniku a vy možná nam…",1
"""Takže rozumím, pane poslanče, prostřednictvím paní předsedající, že jediné, co pětikoalici žene dopředu, je pomsta. Takže jediné, když jsme my nezvoli…",1
"""Dobré dopoledne. Já bych chtěl reagovat na výstup pana poslance Adamce a chtěl bych mu říct, že my jsme neustále obviňováni, že hlasujeme jako ovce. T…",1
"""Ano, ten druhý pozměňovací návrh, který se týká BIS, bere peníze z vládní rozpočtové rezervy, ale jak říkám, byla na něm i politická shoda na výboru p…",1
"""Vážený pane předsedající, kolegyně, kolegové, já bych položil jenom dva krátké dotazy. Pokud se tu Piráti zaštiťují tím dodržováním zákonů, jaký je te…",1
"""Děkuji, pane předsedající. Já bych krátce zareagoval na pana kolegu Lochmana. Mě vážně strašně mrzí, že jste tu nebyl v minulém volebním období, mě to…",1
"""Nyní se přesuneme na začátek roku 2023. S velkou pompou se mluví o tom, že MPSV pracuje na důchodové reformě. Na veřejnost začínají prosakovat informa…",1
